In [10]:
import json
import requests
import sys
import arxivpy
from google.cloud import translate

# google翻訳は一旦なしでやるヨロシ：田口
translate_client = translate.Client.from_service_account_json('/Users/01017387/Desktop/google_credentials/translate-a0f64139d4a6.json')
sys.path.append("/Users/01017387/Desktop/git関連/websocket-client")
from slackclient import SlackClient

# slackAPIは別途渡します：田口
file = "/Users/01017387/.credentials/slack_apikey"
f = open(file,'r')
for row in f:
    slack_token = row.replace("\n","")
f.close()

sc = SlackClient(slack_token)
list_url = "https://slack.com/api/channels.list"

# activeなチャンネル名を取得
active_channels = sc.api_call(
  "channels.list"
)

# slack投稿する対象のチャンネル名を特定
channel_names = [c["name"] for c in active_channels["channels"]]
remove_target = ['general', 'random', 'slack_apiに必要なもの', 'test_arxiv', 'アイデア']
for r in remove_target:
    channel_names.remove(r)
# 保存しているチャンネルと比較して、新しい物が追加されている場合は処理対象に追加する

res = 2

# チャンネル名の分だけ投稿
for c in channel_names:
    #結果検索部分
    papers = arxivpy.query(c,max_index=res, sort_by="submittedDate", sort_order="descending")
    pdf_url, summary, title, published = [],[],[],[]
    exec_target = [pdf_url, summary, title, published]
    for i in papers:
        title.append([i["title"]])
        #abstract翻訳部分
        text = i["abstract"]
        target = 'ja'
        translation = translate_client.translate(text, target_language=target)['translatedText']
        summary.append([translation])

        published.append([i["publish_date"]])
        pdf_url.append([i["pdf_url"]])
    
    #slack通知部分
    exec_target = [title, summary, pdf_url, published]
    for t,s,pd,p in zip(title, summary, pdf_url, published):
        sc.api_call("chat.postMessage", channel=c,text=t)
        sc.api_call("chat.postMessage", channel=c,text=str(s))
        sc.api_call("chat.postMessage", channel=c,text=pd)
        sc.api_call("chat.postMessage", channel=c,text=p[0].strftime("%Y-%m-%d %H:%M:%S"))